문제 생성기

원본: https://github.com/ECuiDev/obsidian-quiz-generator

In [45]:
import ollama
import json

In [46]:
user_content = """
그러자 어른들은 내게 충고하길 엉뚱한 보아 뱀이나 그리지 말고 지리, 역사, 샘(계산)나 문법에 취미를 들여보래. 그리 하여 난 여섯 날에 화가의 꿈을 접어야 했지. 내 첫 그림과 두 번째 그림이 영 쓸모 없자 낙담하고 말았거든. 나의 이런 일 들에 대해 어른들은 전혀 관심도 없었지. 설명을 해대는 아인 피곤하다는 투였으니.
   그래서 다른 직업을 선택하게 된 거야. 그게 하늘을 나는 비행사지. 난 정말 전 세계를 날아다녔어. 그리 되니 지리학도 좀 도움이 되데. 난 중국이나 애리조나(미국의 주 이름)도 한눈에 첫 보면 알았지. 밤에 길을 잃었을 때도 지리학에 대한 앎 이 도움이 되었고 말이야.
   살아오며 다양한 사람들을 만났어, 대갠 심각한 어른들이었지. 난 그들 사이에 살아야 했으니깐. 그럼 난 그들에게 내 그림을 보여주었단다. 내 의견을 개진하지 않고 말이야.
   명석해 보이는 이들을 만날 때면 난 아끼며 보관해오던 내 첫 그림을 그분들께 보여주었지. 그들이 정말로 이해할까 알 고 싶었거든. 하지만 대답은 항상 이랬어. 모자군요. 그럼 난 보아 뱀 얘긴 꺼내지도 않았지 물론 숲 얘기나 별 얘기도 하지 않았어. 난 그들에게 내 자신을 맞추며, 기껏 '브리지'(카드 게임의 일종), 골프, 정치 그리고 술에 관한 얘기만 했을 뿐이 야. 그럼 어른들은 죄다 날 합리적인 사람이라 말하며 기뻐들 했지.
   """

In [47]:
client = ollama.Client()

In [48]:
original_prompt = """
You are an assistant specialized in generating exam-style questions and answers.
Your response must only be a JSON object with the following property:\n
"questions": An array of JSON objects, where each JSON object represents a question and answer pair.
Each question type has a different JSON object format.\n\n
The JSON object representing true or false questions must have the following properties:\n
"question": The question\n"answer": A boolean representing the answer\n
For example, if I ask you to generate true or false question, the structure of your response should look like this:\n
{"question": "HTML is a programming language.", "answer": false}
\n\n
The generated questions and answers must be in Korean. However, your 
response must still follow the JSON format provided above. This means that while the values should 
be in Korean, the keys must be the exact same as given above, in English.
"""
original_user_prompt = f"""
Generate true or false question about the following text:\n
{user_content}
\n\nIf the above text contains LaTeX, you should use $...$ (inline math mode) for mathematical symbols.
The overall focus should be on assessing understanding and critical thinking.
"""

In [49]:
my_system_prompt = """
당신은 시험문제 형식의 질문과 답을 생성하는 전문가입니다. 
당신은 다음의 JSON Object 형식으로만 답해야 합니다: 
"questions": JSON Object 배열입니다. 각각의 JSON Object는 'question'과 'answer'가 쌍으로 있어야 합니다.
각 질문 타입은 다른 JSON Object 형식을 가집니다.\n\n
참 거짓에 대한 질문들은 다음과 같은 속성을 따라야 합니다.
"question": 질문\n"answer": 답을 나타내는 참거짓\n
예를 들어, 만약 내가 참 거짓에 대한 문제를 요구한다면, 당신의 답은 다음과 같은 형식을 가져야 합니다:\n
{"question": "HTML은 프로그래밍 언어이다.", "answer": false}
\n\n
생성한 문제들과 답변들은 한국어여야만 합니다. 그럼에도 당신은 위의 JSON 형식을 따라서 답변해야 합니다. 이 말은 값들은 한국어여야 하고, 키는 위에 주어진 영어의 것과 동일해야 합니다.
"""
my_user_prompt = f"""
참과 거짓에 대한 문제를 다음의 텍스트를 따라 작성하세요:\n
{user_content}
\n\n만약 위 텍스트가 LaTeX를 포함한다면, 당신은 $...$ (inline math mode)로 수학 기호를 사용해야 합니다.
이 중점은 이해를 증진시키고 생각하는데 있습니다. 
"""

In [50]:
aya_system_prompt = """
<aya-ex>당신은 시험 문제와 답변을 생성하는 데 특화된 보조 기능을 갖춘 전문가입니다.
당신의 답변은 다음 속성을 가진 JSON 객체여야 합니다:\n
"questions": JSON 배열로, 각 JSON 객체는 질문과 답변 쌍을 나타냅니다.
각 질문 유형은 다른 JSON 객체 형식으로 표현됩니다.\n\n
진실 또는 거짓 질문을 나타내는 JSON 객체는 다음과 같은 속성을 가져야 합니다:\n
"question": 질문 텍스트\n"답변": 답변을 나타내는 부울(boolean) 값\n
예를 들어, 진실 또는 거짓 질문을 생성해 달라고 요청한다면, 답변 구조는 다음과 같아야 합니다:\n
{"question": "HTML is a programming language.", "answer": false}
\n\n
생성된 질문과 답변은 한국어로 작성되어야 합니다. 그러나 제공된 위의 JSON 형식에 맞춰야 합니다. 이는 값이 한국어여도 키는 위에서 지정한 영어 그대로 유지해야 함을 의미합니다.
"""
aya_user_prompt = f"""
제공된 텍스트에 대한 진실 또는 거짓 질문을 생성하세요:\n
{user_content}
\n\n제공된 텍스트에 라텍스(LaTeX)가 포함된 경우, 수식 기호를 위해 `$...$` (인라인 수학 모드)를 사용해야 합니다.
전반적인 초점은 이해도와 비판적 사고력을 평가하는 데 맞춰져야 합니다.
"""

In [51]:
system_prompt = """
<내 번역>당신은 시험문제 형식의 질문과 답을 생성하는 전문가입니다. 
<aya-ex>당신은 시험 문제와 답변을 생성하는 데 특화된 보조 기능을 갖춘 전문가입니다.
You are an assistant specialized in generating exam-style questions and answers.

당신은 다음의 JSON Object 형식으로만 답해야 합니다: 
당신의 답변은 다음 속성을 가진 JSON 객체여야 합니다:
Your response must only be a JSON object with the following property:\n

"questions": JSON Object 배열입니다. 각각의 JSON Object는 'question'과 'answer'가 쌍으로 있어야 합니다.
"questions": JSON 배열로, 각 JSON 객체는 질문과 답변 쌍을 나타냅니다.
"questions": An array of JSON objects, where each JSON object represents a question and answer pair.

각 질문 타입은 다른 JSON Object 형식을 가집니다.
각 질문 유형은 다른 JSON 객체 형식으로 표현됩니다.
Each question type has a different JSON object format.\n\n

참 거짓에 대한 질문들은 다음과 같은 속성을 따라야 합니다.
진실 또는 거짓 질문을 나타내는 JSON 객체는 다음과 같은 속성을 가져야 합니다:
The JSON object representing true or false questions must have the following properties:\n

"question": 질문\n"answer": 답을 나타내는 참거짓
"question": 질문 텍스트\n"답변": 답변을 나타내는 부울(boolean) 값
"question": The question\n"answer": A boolean representing the answer\n

예를 들어, 만약 내가 참 거짓에 대한 문제를 요구한다면, 당신의 답은 다음과 같은 형식을 가져야 합니다:
예를 들어, 진실 또는 거짓 질문을 생성해 달라고 요청한다면, 답변 구조는 다음과 같아야 합니다:
For example, if I ask you to generate true or false question, the structure of your response should look like this:\n

{"question": "HTML은 프로그래밍 언어이다.", "answer": false}
{"question": "HTML is a programming language.", "answer": false}

\n\n
생성한 문제들과 답변들은 한국어여야만 합니다. 그럼에도 당신은 위의 JSON 형식을 따라서 답변해야 합니다. 이 말은 값들은 한국어여야 하고, 키는 위에 주어진 영어의 것과 동일해야 합니다.
생성된 질문과 답변은 한국어로 작성되어야 합니다. 그러나 제공된 위의 JSON 형식에 맞춰야 합니다. 이는 값이 한국어여도 키는 위에서 지정한 영어 그대로 유지해야 함을 의미합니다.
The generated questions and answers must be in Korean. However, your 
response must still follow the JSON format provided above. This means that while the values should 
be in Korean, the keys must be the exact same as given above, in English.
"""

In [52]:
user_prompt = f"""
참과 거짓에 대한 문제를 다음의 텍스트를 따라 작성하세요:
제공된 텍스트에 대한 진실 또는 거짓 질문을 생성하세요.
Generate true or false question about the following text:\n

{user_content}

만약 위 텍스트가 LaTeX를 포함한다면, 당신은 $...$ (inline math mode)로 수학 기호를 사용해야 합니다.
제공된 텍스트에 라텍스(LaTeX)가 포함된 경우, 수식 기호를 위해 `$...$` (인라인 수학 모드)를 사용해야 합니다.
\n\nIf the above text contains LaTeX, you should use $...$ (inline math mode) for mathematical symbols.

이 중점은 이해를 증진시키고 생각하는데 있습니다. 
전반적인 초점은 이해도와 비판적 사고력을 평가하는 데 맞춰져야 합니다.
The overall focus should be on assessing understanding and critical thinking.
"""

In [53]:
model = "EEVE-Korean-10.8B"

In [54]:
result = []
pass_count = 0
for i in range(10):
    response = client.generate(
        model=model,
        system=original_prompt,
        prompt=original_user_prompt,
        format='json'
    )
    res = json.loads(response['response'])
    result.append(res)
    pass_count += 1 if ('question' in res and 'answer' in res and isinstance(res["answer"], bool)) else 0
pass_count, result

(7,
 [{'questions': [{'question': '주인공은 자신의 그림에 대해 어른들로부터 어떤 충고를 받았습니까?',
     'answer': False},
    {'question': '주인공이 화가로서의 꿈을 접게 된 이유는 무엇입니까?', 'answer': True},
    {'question': '주인공은 자신의 첫 번째와 두 번째 그림을 어떻게 묘사했습니까?', 'answer': True},
    {'question': '새로운 직업으로 선택한 비행사는 지리학에 어떤 도움이 되었습니까?', 'answer': True},
    {'question': '주인공은 다양한 사람들을 만났을 때 자신의 그림을 어떻게 공유했습니까?', 'answer': False}]},
  {'question': '해당 텍스트에서 화자가 어릴 때 그림 그리기에 대한 열정을 포기하고 비행기 조종사가 되기로 결정했습니까?',
   'answer': False},
  {'questions': [{'question': '해당 텍스트에서 화자가 처음 꿈을 접어야 했던 이유는 무엇입니까?',
     'answer': False},
    {'question': '화자는 비행기 조종사가 된 후 지리학 지식이 어떻게 도움이 되었습니까?', 'answer': True}]},
  {'question': '해당 텍스트에서 화자가 자신의 첫 번째 그림과 두 번째 그림을 왜 영 쓸모 없다고 여겼습니까?',
   'answer': False},
  {'question': '해당 텍스트에서 해당 인물이 자신의 그림을 다른 사람들과 나누는 데 있어서 첫 번째 그림과 두 번째 그림의 반응에 따라 어떻게 접근했는지 언급했습니까?',
   'answer': False},
  {'question': '해당 텍스트에서 화자는 자신의 그림에 대해 지리, 역사, 계산 또는 문법을 배우는 데 도움이 되었다고 언급했습니까?',
   'answer': False},
  {'question': '본문 

In [55]:
result = []
pass_count = 0
for i in range(10):
    response = client.generate(
        model=model,
        system=my_system_prompt,
        prompt=my_user_prompt,
        format='json'
    )
    res = json.loads(response['response'])
    result.append(res)
    pass_count += 1 if ('question' in res and 'answer' in res and isinstance(res["answer"], bool)) else 0
pass_count, result


(8,
 [{'question': '해당 텍스트에서 화자는 자신의 첫 번째 그림과 두 번째 그림을 영 쓸모 없다고 표현했습니까?',
   'answer': True},
  {'question': '저자가 자신의 첫 번째 그림과 두 번째 그림을 보여준 후에 어른들로부터 어떤 반응을 받았습니까?',
   'answer': "어른들은 저자의 첫 번째와 두 번째 그림을 '부족하다'고 평가했습니다."},
  {'question': '텍스트에 따르면 화자는 처음에 어떤 예술 분야를 추구했습니까?', 'answer': False},
  {'question': '해당 인물은 지리, 역사, 계산 또는 문법을 취미를 들일 것을 권유받았습니까?', 'answer': True},
  {'question': '텍스트에서 화자는 지리, 역사, 샘(계산), 문법에 대한 관심으로 보아 엉뚱한 그림을 그리지 말라고 충고받았습니까?',
   'answer': True},
  {'question': '위 텍스트에 따르면 화자는 어린 시절 자신의 예술적 꿈을 포기하고 다른 경력을 추구했습니까?',
   'answer': True},
  {'question': '해당 인물은 자신의 첫 번째 그림에 대해 부정적인 반응을 받았습니까?', 'answer': False},
  {'question': '본문에 따르면 화자는 어린 시절 자신의 그림을 보여주는 것을 중단한 이유는 무엇인가요?',
   'answer': False},
  {'questions': [{'question': '텍스트에서 주인공은 자신의 첫 번째 그림과 두 번째 그림을 왜 포기했습니까?',
     'answer': False},
    {'question': '주인공이 지리학에 관심을 가지게 된 것은 무엇입니까?', 'answer': True},
    {'question': '텍스트에서 주인공은 누구에게 자신의 첫 번째 그림을 보여주었나요?', 'answer': False}]},
  {'question': '주어진 텍스트에서 화자

In [56]:
result = []
pass_count = 0
for i in range(10):
    response = client.generate(
        model=model,
        system=aya_system_prompt,
        prompt=aya_user_prompt,
        format='json'
    )
    res = json.loads(response['response'])
    result.append(res)
    pass_count += 1 if ('question' in res and 'answer' in res and isinstance(res["answer"], bool)) else 0
pass_count, result

(10,
 [{'question': '해당 글에서 화자가 어린 시절 지리, 역사, 샘(계산), 문법에 대한 관심으로 보아 뱀 그림을 그리지 않았다고 언급했습니까?',
   'answer': False},
  {'question': '텍스트에 따르면 화가가 보아 뱀이나 별과 같은 주제보다는 지리, 역사, 계산, 문법에 더 집중하려고 했습니까?',
   'answer': True},
  {'question': '제공된 텍스트에 따르면, 화가는 첫 번째 그림과 두 번째 그림이 쓸모없다고 판단한 후 지리학이나 역사와 같은 새로운 취미나 직업 분야에 관심을 기울였습니까?',
   'answer': True},
  {'question': '제공된 텍스트에 따르면 화자는 처음에 화가 꿈을 접고 비행사가 되기로 한 이유는 무엇인가요?',
   'answer': False},
  {'question': '텍스트에서 화자가 어린 시절 그림 그리기에 대한 열정을 포기하게 된 이유는 무엇인가요?',
   'answer': False},
  {'question': "주어진 텍스트에 따르면 저자는 자신의 첫 번째 그림과 두 번째 그림을 '쓸모없다'고 느꼈습니까?",
   'answer': True},
  {'question': '텍스트에 따르면 주인공은 어른들이 추천한 취미와 관심사를 따랐습니까?', 'answer': False},
  {'question': '텍스트에 따르면, 화자는 어린 시절 숲이나 별과 같은 주제를 그린 그림을 그리다가 어른들의 충고로 지리, 역사, 수학, 문법에 관심을 가지게 되었나요?',
   'answer': True},
  {'question': '해당 인물이 화가가 되려는 꿈을 접고 비행사가 되기 전에, 지리학이 그들의 직업 선택에 결정적인 역할을 했습니까?',
   'answer': True},
  {'question': '제공된 텍스트에서 화자는 자신의 초기 관심사와 열정이 무시되거나 경시당했다고 언급합니까?',
   'answer': True}])

In [64]:
r = client.generate(
    model='aya-expanse',
    system=f"""
    당신은 문제 검증 전문가이다. <사용자>의 입력이 <지문>과 비교했을 때 올바른 문제들인지 검증하라. 질문과 답변이 올바른지 'verify'에 기록하고, 'reason'에 그 이유를 적어라. 'real_answer'에 실제 정답을 적어라.
    <지문>
    {user_content}
    </지문>
    """,
    prompt=f"{result}",
    format='json'
)
r['response']

'{\n  "verify": [\n    {\n      "question": "해당 글에서 화자가 어린 시절 지리, 역사, 샘(계산), 문법에 대한 관심으로 보아 뱀 그림을 그리지 않았다고 언급했습니까?",\n      "answer": false,\n      "reason": "글에서는 화자가 보아 뱀 그림을 그렸다는 직접적인 언급이 없으며, 오히려 처음에 그린 그림이 쓸모없었다고 표현함."\n    },\n    {\n      "question": "텍스트에 따르면 화가는 보아 뱀이나 별과 같은 주제보다는 지리, 역사, 계산, 문법에 더 집중하려고 했습니까?",\n      "answer": true,\n      "reason": "글에서 화자는 어른들의 충고에 따라 지리, 역사, 샘(계산), 문법 등에 관심을 갖게 되었다고 명시."\n    },\n    {\n      "question": "제공된 텍스트에 따르면, 화가는 첫 번째 그림과 두 번째 그림이 쓸모없다고 판단한 후 지리학이나 역사와 같은 새로운 취미나 직업 분야에 관심을 기울였습니까?",\n      "answer": true,\n      "reason": "글에서 화자가 초기 그림에 낙담하고, 지리학 및 역사를 통해 세계를 이해하게 되었다고 설명."\n    },\n    {\n      "question": "제공된 텍스트에 따르면 화자는 자신의 첫 번째 그림과 두 번째 그림을 \'쓸모없다\'고 느꼈습니까?",\n      "answer": true,\n      "reason": "글에서 화자가 첫 그림과 두 번째 그림이 쓸모없다고 명시적으로 언급."\n    },\n    {\n      "question": "텍스트에 따르면 주인공은 어른들이 추천한 취미와 관심사를 따랐습니까?",\n      "answer": false,\n      "reason": "화자는 어른들의 충고를 따르긴 했지만, 자신의 초기 열정을 완전히 포기하지는 않았음. 글 후반부에서는 자신의 그림을 보여

In [65]:
print(json.dumps(json.loads(r['response']), indent=4, sort_keys=True, ensure_ascii=False))

{
    "real_answer": [
        {
            "answer": false,
            "question": "해당 글에서 화자가 어린 시절 지리, 역사, 샘(계산), 문법에 대한 관심으로 보아 뱀 그림을 그리지 않았다고 언급했습니까?"
        },
        {
            "answer": true,
            "question": "텍스트에 따르면 화가는 보아 뱀이나 별과 같은 주제보다는 지리, 역사, 계산, 문법에 더 집중하려고 했습니까?"
        },
        {
            "answer": true,
            "question": "제공된 텍스트에 따르면, 화가는 첫 번째 그림과 두 번째 그림이 쓸모없다고 판단한 후 지리학이나 역사와 같은 새로운 취미나 직업 분야에 관심을 기울였습니까?"
        },
        {
            "answer": "글 전반부에서 화자의 낙담과 어른들의 충고에 대한 언급으로 미루어 보아, 어른들의 권유와 자신의 그림에 대한 실망감이 원인일 수 있음.",
            "question": "텍스트에서 화자가 어린 시절 그림 그리기에 대한 열정을 포기하게 된 이유는 무엇인가요?"
        },
        {
            "answer": false,
            "question": "텍스트에 따르면 주인공은 어른들이 추천한 취미와 관심사를 따랐습니까?"
        },
        {
            "answer": true,
            "question": "해당 인물이 화가가 되려는 꿈을 접고 비행사가 되기 전에, 지리학이 그들의 직업 선택에 결정적인 역할을 했습니까?"
        },
        {
            "answer": true,
            "question": "제공된 텍스트에서 화자는 자신의